In [19]:
import torch
dataset_name = 'en'
task_embedding = torch.load(f'outputs/{dataset_name}/task_embedding.pt')
data_embedding = torch.load(f'outputs/{dataset_name}/all_data_task_embedding.pt')

In [20]:
import pandas as pd
import os
task_list = pd.read_csv(f'../data/{dataset_name}/task.csv', sep='\t')['task'].tolist()
job_posting = pd.read_csv(f'outputs/{dataset_name}/all_data_task.csv', sep='\t')

In [21]:
import torch

def cosine_similarity_matrix(x, y):
    x_norm = x / x.norm(dim=1, keepdim=True)
    y_norm = y / y.norm(dim=1, keepdim=True)
    return torch.mm(x_norm, y_norm.t())

x = torch.rand(5, 128)  
y = torch.rand(6, 128)
similarity_matrix = cosine_similarity_matrix(x, y)
print(similarity_matrix)

tensor([[0.7721, 0.8061, 0.7319, 0.7623, 0.7213, 0.7845],
        [0.7398, 0.7593, 0.7585, 0.7467, 0.7450, 0.7608],
        [0.7833, 0.7445, 0.7656, 0.7475, 0.7361, 0.7506],
        [0.7877, 0.7112, 0.7012, 0.7821, 0.7834, 0.7820],
        [0.7574, 0.7360, 0.7364, 0.7336, 0.6967, 0.7246]])


In [22]:
similarity_matrix = cosine_similarity_matrix(data_embedding, task_embedding)
topk_indices = torch.topk(similarity_matrix, dim=-1,k=10).indices
topk_value = torch.topk(similarity_matrix, dim=-1,k=10).values

In [23]:
import pandas as pd
topk_task = pd.DataFrame(topk_indices.numpy())
topk_values = pd.DataFrame(topk_value.numpy())

In [24]:
for col in topk_task.columns:
    job_posting[f"task{col}"] = topk_task[col].apply(lambda x: task_list[x]) 
    job_posting[f"task{col}_sim"] = topk_values[col] 
job_posting.to_csv(f'outputs/{dataset_name}/topk_task.csv', index=None, sep='\t')
job_posting

,occupation,text,task,task0,task0_sim,task1,task1_sim,task2,task2_sim,task3,...,task5,task5_sim,task6,task6_sim,task7,task7_sim,task8,task8_sim,task9,task9_sim
0,Real Estate,Job descriptionA leading real estate firm in N...,Agent Assistance- Receive & Organize Marketing...,Write interesting and effective press releases...,0.846239,Write interesting and effective press releases...,0.840311,"Prepare promotional plans, sales literature, m...",0.837001,Investigate products or markets to determine a...,...,Prepare or organize online storefront marketin...,0.832986,Evaluate new emerging media or technologies an...,0.831872,Prepare and deliver sales presentations to new...,0.831091,"Identify prospective customers, using business...",0.826410,"Conduct online marketing initiatives, such as ...",0.823900
1,Wellness and Fitness Services,"At Aspen Therapy and Wellness , we are committ...","At Aspen Therapy and Wellness , we are committ...",Encourage patients and attend to their physica...,0.831715,Work with clients to identify their financial ...,0.822980,Counsel children and families to help solve co...,0.822472,Maintain and promote a positive attitude towar...,...,Collaborate with other health care professiona...,0.812278,Advocate for accessible health care that minim...,0.810266,Educate patients about maintenance and promoti...,0.808610,Plan and direct physical conditioning programs...,0.806425,Adapt teaching methods and instructional mater...,0.806025
2,Wellness and Fitness Services,"At Aspen Therapy and Wellness , we are committ...",We are actively looking to hire a therapist in...,Encourage patients and attend to their physica...,0.813300,Counsel children and families to help solve co...,0.807270,Educate patients about maintenance and promoti...,0.806549,Maintain and promote a positive attitude towar...,...,Work with clients to identify their financial ...,0.804653,Meet regularly with parents to discuss childre...,0.804106,Network within communities to find and attract...,0.800965,Apply current technology to music therapy prac...,0.800471,Collaborate with other educational professiona...,0.800388
3,Law Practice,Senior Associate Attorney - Elder Law / Trusts...,Senior Associate Attorney - Elder Law / Trusts...,Provide customers with ongoing technical support,0.823697,Communicate with counsel regarding case manage...,0.818487,Maintain and promote a positive attitude towar...,0.817481,Maintain knowledge of community services and r...,...,Consult with corporate attorneys as necessary ...,0.813881,Guide clients in the development of skills or ...,0.812122,Explain to clients the personal financial advi...,0.811306,Refer students to outside counseling services,0.810925,Present clients with information required to m...,0.810458
4,Hospitals and Health Care,"At Children’s, the region’s only full-service ...",Children's is committed to diversity and inclu...,Coordinate placement of students with special ...,0.781886,Meet regularly with parents to discuss childre...,0.777870,Keep informed about trends in education and su...,0.776891,Involve parent volunteers and older students i...,...,Counsel children and families to help solve co...,0.764388,Teach social skills to students,0.753481,Support children's emotional and social develo...,0.753312,Help prepare and serve nutritionally balanced ...,0.749653,"Keep students informed of community events, su...",0.741335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10897,Machinery Manufacturing,Are you a dynamic and experienced Hydraulic Sy...,"Through comprehensive training, meaningful rec...",Plan and direct physical conditioning programs...,0.831307,Adapt teaching methods and instructional mater...,0.827532,Collaborate with other marketing staff to inte...,0.824412,Participate in continuing education activities...,...,Collaborate with others in the organization to...,0.818359,Collaborate with other health care professiona...,0.816882,Collaborate with other educ

In [25]:
sample_k = 50
df = job_posting.sort_values('task0_sim', ascending=False)[:sample_k].drop('text', axis=1)
df.to_excel(f'outputs/{dataset_name}/sample_top{sample_k}.xlsx')
df

,occupation,task,task0,task0_sim,task1,task1_sim,task2,task2_sim,task3,task3_sim,...,task5,task5_sim,task6,task6_sim,task7,task7_sim,task8,task8_sim,task9,task9_sim
8884,Construction,Conduct project/program deliverable prioritiza...,Communicate with key stakeholders to determine...,0.920136,Consult with management to ensure agreement on...,0.917694,Prepare reports and document project activitie...,0.916462,Monitor project milestones and deliverables,0.911370,...,Review project plans to plan and coordinate pr...,0.910891,"Manage own accounts and projects, working with...",0.910828,Oversee project production efforts to assure p...,0.910716,Research and analyze member or community needs...,0.910497,Draw up detailed specifications and discuss pr...,0.907054
9667,Non-profit Organizations,Identify and coordinate staff training needs a...,Initiate sales campaigns to meet sales and pro...,0.915956,Identify and recruit potential volunteer workers,0.914985,Identify training and development needs,0.914770,Monitor compliance systems to ensure their eff...,0.912407,...,Document clients' progress toward meeting esta...,0.911082,Identify skill development needs for funeral h...,0.910898,Research and analyze member or community needs...,0.908099,Establish work schedules and assign work to st...,0.907506,Monitor sales staff performance to ensure that...,0.907149
4714,Civic and Social Organizations,Mentor and guide other staff who are deeply co...,Maintain and promote a positive attitude towar...,0.909605,Teach instructional courses and advise students,0.902517,Develop or teach strategies to promote client ...,0.901594,Coach client representatives in effective comm...,0.900181,...,Document clients' progress toward meeting esta...,0.894635,Plan and conduct new-employee orientations to ...,0.894098,Coach senior executives and managers on leader...,0.892950,Develop individualized educational plans in co...,0.892729,Plan and conduct new employee orientation to f...,0.892713
6692,Education Administration Programs,"At CDS, our School Principals are deeply commi...","Establish clear objectives for all lessons, un...",0.907558,"Establish clear objectives for all lessons, un...",0.906102,Plan and conduct new-employee orientations to ...,0.903158,Adapt teaching methods and instructional mater...,0.898987,...,Collaborate with teachers to develop and maint...,0.895369,"Establish clear objectives for all lessons, un...",0.894368,"Formulate and implement training programs, app...",0.894110,Plan physical education programs to promote de...,0.894046,Administer standardized ability and achievemen...,0.891159
7110,Machinery Manufacturing,Lead and manage all business and/or department...,Plan and conduct new-employee orientations to ...,0.906404,Plan and conduct new employee orientation to f...,0.902385,Oversee project production efforts to assure p...,0.895021,Conduct meetings with staff to discuss product...,0.894819,...,Develop staff service or retail goals and guid...,0.894448,Maintain and promote a positive attitude towar...,0.894106,Manage timely flow of business intelligence in...,0.893371,Assign duties to radiologic staff to maintain ...,0.892787,Consult with management to ensure agreement on...,0.892432
7916,Banking,Initiate implementation requests for new busin...,Investigate available investment opportunities...,0.905827,"Direct daily operations of department, analyzi...",0.905433,Identify resale opportunities and support them...,0.901291,"Manage own accounts and projects, working with...",0.899461,...,Analyze acquisitions to ensure conformance wit...,0.894386,Assign duties to radiologic staff to maintain ...,0.892911,Prepare management reports defining and evalua...,0.890669,Provide customers or installation teams guidel...,0.888971,Devise debt liquidation plans that include pay...,0.888713
8003,Software Development,Experience taking a lead role developing compl...,Provide customers or installation teams guidel...,0.904492,Discuss security solutions w